# Previous Version

In [3]:
import millenium
import os
from tqdm import tqdm
import time

os.chdir("/home/mdbai/telarKG/")
PROJECT_DIR = os.getcwd()

In [4]:
# mapping telarkg property graph to rdf
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF, XSD

import json



In [17]:
telar_server = millenium.ML(port=8020)

Exception ignored in: <function ML.__del__ at 0x7f7ac76a20e0>
Traceback (most recent call last):
  File "/home/mdbai/telarKG/RDF/telarKG/millenium.py", line 94, in __del__
    self.shutting_down()
  File "/home/mdbai/telarKG/RDF/telarKG/millenium.py", line 90, in shutting_down
    os.killpg(self.pid, signal.SIGINT)
ProcessLookupError: [Errno 3] No such process


~/MDB-Compatibilidad/MillenniumDB-Dev/build/Release/bin


Initializing server...
-------------------------------------
Catalog:
  identifiable nodes count: 20337308
  anonymous nodes count:    0
  connections count:        2675665
  label count:              20477066
  properties count:         265586131
  distinct labels:          28
  distinct from's:          572591
  distinct to's:            451198
  distinct type's:          23
  distinct keys:            117
  equal_from_to_count:      0
  equal_from_type_count:    0
  equal_to_type_count:      0
  equal_from_to_type_count: 0
-------------------------------------
Server running on port 8020
To terminate press CTRL-C


In [6]:
# load node n edge attributes from json
with open(PROJECT_DIR + "/data/node_attribute.json", "r") as file:
    node_attrs = json.load(file)

with open(PROJECT_DIR + "/data/edge_type_attribute.json", "r") as file:
    edge_attrs = json.load(file)

In [7]:
def get_instances_from_json(label: str):
    with open(PROJECT_DIR + f"/data/Nodes/{label}_instances.json", "r") as file:
        results = json.load(file)

    return [instance["?instance"] for instance in results["results"]["bindings"]]

In [8]:
def get_describe_from_list(query_list: list):
    for i in query_list:
        with open(".tmp_describe.txt", "w") as file:
            query = f"DESCRIBE PROPERTIES INCOMING {i}"
            file.write(query)
        telar_server.run_to_file(
            test_file=".tmp_describe.txt", output_file=".tmp_query.json"
        )

        with open(".tmp_query.json", "r") as file_temp:
            output = json.load(file_temp)

    return output

In [9]:
# load labels from file
telar_node_labels = []
with open(os.path.join(PROJECT_DIR, "data/node_labels.txt"), "r") as file:
    lines = file.readlines()
    for line in lines:
        telar_node_labels.append(line.strip('",\n'))

In [10]:
telar_node_labels

['ConventionMember',
 'Chat',
 'Pact',
 'CommissionMaterial',
 'TrendingTopic',
 'Topic',
 'Transcription',
 'Tweet',
 'Speech',
 'FacebookPost',
 'Video',
 'Party',
 'Candidate',
 'VoteResult',
 'TwitterAccount',
 'TypeContent',
 'FacebookAccount',
 'Intervention',
 'District',
 'PlenaryMaterial',
 'PoliticalGroupConvention',
 'InstagramPost',
 'Commission',
 'PlenarySession',
 'CommissionSession',
 'Gender',
 'InstagramAccount',
 'Language']

In [25]:
instances = get_instances_from_json("PlenaryMaterial")

In [13]:
sample = get_describe_from_list([instances[0]])

---------------------------------------
 Query received:
---------------------------------------
DESCRIBE PROPERTIES INCOMING pers_arancibia_jorge
---------------------------------------
OpDescribe(PROPERTIES INCOMING pers_arancibia_jorge)


Plan Executed:
Describe()
Results: 4302


In [36]:
sample

{'properties': {'age': 81,
  'c': 'I',
  'charge': '2002-2010;S;8803',
  'name': 'Jorge Arancibia',
  'orden': 150,
  'photo': True,
  'photoId': 23280,
  'web': 'arancibiaconstituyente.cl',
  'zone': 60070,
  'inVoteName': 'JORGE PATRICIO ARANCIBIA REYES',
  'occupation': 'Ingeniero en Armas',
  'description': 'Ex senador, fue comandante en jefe de la Armada y embajador en Turquía.',
  'elections': '2001;S;8803;38,35;G',
  'patrimony': 5785930,
  'pact2409': 'Vamos por Chile',
  'nominateNames': 'arancibia_jorge',
  'nombreEnPartido': 'arancibia, jorge',
  'wikidata': 'http://www.wikidata.org/entity/Q5934487'},
 'incoming': [{'node': 'matp_3194', 'type': 'vote', 'edge': '_e1461572'},
  {'node': 'matp_2183', 'type': 'vote', 'edge': '_e1462027'},
  {'node': 'matp_2201', 'type': 'vote', 'edge': '_e1462178'},
  {'node': 'matp_2189', 'type': 'vote', 'edge': '_e1462333'},
  {'node': 'matp_2190', 'type': 'vote', 'edge': '_e1462488'},
  {'node': 'matp_2194', 'type': 'vote', 'edge': '_e1462643

In [7]:
telar_server.shutting_down()

server shutdown



Server shutting down


In [12]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF, XSD

In [13]:
g = Graph()
g.parse('telarKG/RDF/telarKG/telarKG_schema.ttl', format='ttl')

<Graph identifier=N0623889ae2d945f49c604a92e2a4569f (<class 'rdflib.graph.Graph'>)>

In [14]:
print(os.getcwd())

/home/mdbai/telarKG


In [30]:
# get mapping from json config
with open("RDF/telarKG/mapping_node_attribute_config.json", "r") as file:
    config = json.load(file)

In [23]:
print (config['ConventionMember']['age'])

:age


In [31]:
# save triples to file
label = 'PlenaryMaterial'
with open("RDF/telarKG/instances.ttl", "a") as file:

    for inst in instances:
        sample = get_describe_from_list([inst])

        uri_entity = '<https://telarkg.imfd.cl/entity/'+ inst + '>'

        file.write(uri_entity + ' a ' + config[label]['self'] + ' .\n')

        for i in sample['properties']:
            property = config[label][i][0]
            data_type = config[label][i][1]
            value = str(sample['properties'][i])
            separator = "\"" if data_type == '^^xsd:string' else ''

            # discard instance if property = 0
            if property == 0:
                continue

            if data_type == 'xsd:boolean':
                data_type = ''
                value = value.lower()
            object = separator + value + separator + data_type

            file.write (uri_entity + ' ' + property + ' ' + object + ' .\n')

---------------------------------------
 Query received:
---------------------------------------
DESCRIBE PROPERTIES INCOMING matp_3194
---------------------------------------
OpDescribe(PROPERTIES INCOMING matp_3194)


Plan Executed:
Describe()
Results: 18
---------------------------------------
 Query received:
---------------------------------------
DESCRIBE PROPERTIES INCOMING matp_3294
---------------------------------------
OpDescribe(PROPERTIES INCOMING matp_3294)


Plan Executed:
Describe()
Results: 18
---------------------------------------
 Query received:
---------------------------------------
DESCRIBE PROPERTIES INCOMING matp_3835
---------------------------------------
OpDescribe(PROPERTIES INCOMING matp_3835)


Plan Executed:
Describe()
Results: 18
---------------------------------------
 Query received:
---------------------------------------
DESCRIBE PROPERTIES INCOMING matp_2183
---------------------------------------
OpDescribe(PROPERTIES INCOMING matp_2183)


Plan E